In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
dataset_path = 'datasets'

In [2]:
def get_data(train_path, dev_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """

    train_df = pd.read_json(train_path, lines=True)
    val_df = pd.read_json(dev_path, lines=True)
    # test_df = pd.read_json(test_path, lines=True)

    return train_df, val_df

train_df, valid_df = get_data(train_path=os.path.join(dataset_path, "train.jsonl"),dev_path=os.path.join(dataset_path, "val.jsonl"),test_path=None, random_seed=42)

In [3]:
# Remove rows with NaN values in the 'text' column
train_df = train_df.dropna(subset=['text'])
valid_df = valid_df.dropna(subset=['text'])
# show the number of rows and columns in the dataframe
print(f"Train DataFrame shape: {train_df.shape}")
print(f"Validation DataFrame shape: {valid_df.shape}")
# show the first few rows of the dataframe
print("Train DataFrame:")
print(train_df.head())
print("Validation DataFrame:")
print(valid_df.head())

Train DataFrame shape: (23707, 5)
Validation DataFrame shape: (3589, 5)
Train DataFrame:
                                     id  \
0  ea468d03-1973-5039-86b2-ff225bb92c4e   
1  0d05f269-6d67-521d-9b5d-cc18f482c6c1   
2  c2ec79f3-da80-58f8-bef0-3e0ea7ab072f   
3  4ad37c58-0bb7-536b-997d-cfccabd0d094   
4  07747b0c-5051-5e0d-8096-b4d4ed8bd98e   

                                                text  \
0  Duke Ellington, a titan of jazz, revolutionize...   
1  I reflected on the shifting dynamics of media ...   
2  In F. Scott Fitzgerald's "The Great Gatsby," t...   
3  I still chuckle when I think about that time I...   
4  Yoga, originating in ancient India, encompasse...   

                          model  label   genre  
0          falcon3-10b-instruct      1  essays  
1                       o3-mini      1  essays  
2                        gpt-4o      1  essays  
3  deepseek-r1-distill-qwen-32b      1  essays  
4              gemini-2.0-flash      1  essays  
Validation DataFrame:

In [4]:
# 1. 准备数据（假设 train_df 和 test_df 已在内存中）
X_train = train_df['text'].fillna("").tolist()
y_train = train_df['label'].tolist()

X_val = valid_df['text'].fillna("")
y_val = valid_df['label']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, f1_score
from sklearn.model_selection import train_test_split


# 创建 pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 1),
        sublinear_tf=True,
        use_idf=False,
        stop_words='english'
    )),
    ('clf', CalibratedClassifierCV(
        estimator=LinearSVC(
            C=1.0, # 线性SVM的正则化参数
            class_weight='balanced',
            loss='squared_hinge',
            dual=False,
            max_iter=1000,
            random_state=42
        ),
        method='sigmoid',  # 或 'isotonic'，通常 sigmoid 更快
        cv=3  # 内部交叉验证用于拟合概率
    ))
])

# 训练模型
pipeline.fit(X_train, y_train)

TypeError: CalibratedClassifierCV.__init__() got an unexpected keyword argument 'base_estimator'